In [10]:
# Require packages 
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import sample
from cobra.flux_analysis import flux_variability_analysis
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from scipy.stats import linregress
import pandas as pd

In [11]:
ANmodel = cobra.io.read_sbml_model("ArabidopsisCoreModel1.xml")

# Changing the directionality of Fum_c to Mal_c to go the other way (day-time model) 
r = ANmodel.reactions.get_by_id('FumHA_c')
r.add_metabolites({ANmodel.metabolites.get_by_id('Fum_c'): +2})
r.add_metabolites({ANmodel.metabolites.get_by_id('H2O_c'): +2})
r.add_metabolites({ANmodel.metabolites.get_by_id('Mal_c'): -2})
print(r.reaction)
print(r.bounds)
print("Directionality changed. Dyson et al. 2016 \n")

# Creating a Malate "Storage" Reactions
r = cobra.Reaction('Mal_Store')
r.name = 'Mal_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('Mal_c'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

# Creating a Fumarate "Storage" Reactions
r = cobra.Reaction('Fum_Store')
r.name = 'Fum_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('Fum_c'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

# Creating a Starch "Storage" Reactions
r = cobra.Reaction('Starch_Store')
r.name = 'Starch_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('starch1_h'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f05c5d5a390>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f05c5dcc7f0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f05c5d5ae10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 


In [8]:
for m in ['Arg_h']:
    metabo = ANmodel.metabolites.get_by_id(m)
    for r in metabo.reactions:
        print(r)
    print(" ")

Tr_Arg2: Arg_h --> Arg_c
Ex_Arg_h: Arg_h --> 
ArgSCAL_h: Arg_DASH_SCA_h --> Arg_h + Fum_h
 


In [12]:
rMal = ANmodel.reactions.get_by_id('Mal_Store')
rFum = ANmodel.reactions.get_by_id('Fum_Store')
rFum.upper_bound = 0.0
rFum.lower_bound = 0.0
rStarch = ANmodel.reactions.get_by_id('Starch_Store')
rCO2 = ANmodel.reactions.get_by_id('Im_CO2')
rO2 = ANmodel.reactions.get_by_id('Ex_O2')
rArg = ANmodel.reactions.get_by_id('ArgSCAL_h')
rSCF = ANmodel.reactions.get_by_id('Tr_SFC')
FumHA_c = ANmodel.reactions.get_by_id('FumHA_c')

rObj = ANmodel.reactions.get_by_id("Ex_O2")
print(rObj.reaction)
ANmodel.objective = rObj
ANmodel.objective_direction = "Minimum"
print(ANmodel.objective)
allreacs = [rObj,rMal,rFum,rStarch,rCO2,rArg,]

f1=0.99*200
f2=1.01*200

f1=200
f2=200

# # Col0 Control
# # rO2.upper_bound = 49.733333
# # rO2.lower_bound = 49.733333
# rCO2.upper_bound = 101.0
# rCO2.lower_bound = 99.0
# rMal.upper_bound = 0.004*f2
# rMal.lower_bound = 0.002*f1
# rFum.upper_bound = 0.007*f2
# rFum.lower_bound = 0.005*f1
# rStarch.upper_bound = 0.029*f2
# rStarch.lower_bound = 0.027*f1
# cobra.io.write_sbml_model(ANmodel,"ANmodel_Col0_Control2.xml",use_fbc_package=False)
# print(flux_variability_analysis(ANmodel, allreacs))

# # Col0 Cold
# # rO2.upper_bound = 54.343333
# # rO2.lower_bound = 54.343333
# rCO2.upper_bound = 101.0
# rCO2.lower_bound = 99.0
# rMal.upper_bound = 0.013*f2
# rMal.lower_bound = 0.011*f1
# rFum.upper_bound = 0.024*f2
# rFum.lower_bound = 0.022*f1
# rStarch.upper_bound = 0.052*f2
# rStarch.lower_bound = 0.05*f1
# cobra.io.write_sbml_model(ANmodel,"ANmodel_Col0_Cold2.xml",use_fbc_package=False)
# print(flux_variability_analysis(ANmodel, allreacs))

# Fum2 Control
# rO2.upper_bound = 52.29
# rO2.lower_bound = 52.29
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
# rMal.upper_bound = 0.013*f2
# rMal.lower_bound = 0.011*f1
# rFum.upper_bound = 0.0*f2
# rFum.lower_bound = 0.0*f1
# rStarch.upper_bound = 0.043*f2
# rStarch.lower_bound = 0.041*f1
cobra.io.write_sbml_model(ANmodel,"ANmodel_Fum2_Control2.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))

# Fum2 Cold
# rO2.upper_bound = 52.373333
# rO2.lower_bound = 52.373333
rCO2.upper_bound = 95.0
rCO2.lower_bound = 93.0
# rMal.upper_bound = 0.026*f2
# rMal.lower_bound = 0.024*f1
# rFum.upper_bound = 0.0*f2
# rFum.lower_bound = 0.0*f1
# rStarch.upper_bound = 0.064*f2
# rStarch.lower_bound = 0.062*f1
cobra.io.write_sbml_model(ANmodel,"ANmodel_Fum2_Cold2.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))


O2_c --> 
Minimize
-1.0*Ex_O2_reverse_09189 + 1.0*Ex_O2
                   maximum    minimum
ArgSCAL_h    -9.497193e-14   0.000000
Ex_O2         1.250000e+02  68.322857
Fum_Store     0.000000e+00   0.000000
Im_CO2        1.010000e+02  99.000000
Mal_Store     2.525000e+01   0.000000
Starch_Store  1.683333e+01   0.000000
                   maximum       minimum
ArgSCAL_h    -3.599798e-15 -2.138039e-16
Ex_O2         1.250000e+02  6.340286e+01
Fum_Store     0.000000e+00  0.000000e+00
Im_CO2        9.500000e+01  9.300000e+01
Mal_Store     2.375000e+01  0.000000e+00
Starch_Store  1.583333e+01  0.000000e+00


In [7]:
for m in ['Fum_c','Mal_c',"starch1_h","starch3_h","starch5_h"]:
    metabo = ANmodel.metabolites.get_by_id(m)
    for r in metabo.reactions:
        print(r.reaction)
    print(" ")

Mal_c --> Fum_c + H2O_c
Fum_c --> 
Fum_m + SCA_c --> Fum_c + SCA_m
Fum_h --> Fum_c
 
H_c + Mal_c + NADP_c --> CO2_c + NADPH_c + Pyr_c
Mal_c --> Fum_c + H2O_c
A_DASH_CoA_c + GLX_c + H2O_c --> CoA_c + H_c + Mal_c
Mal_c + iCit_m <=> Mal_m + iCit_c
Mal_c --> 
Mal_c + Pi_m <=> Mal_m + Pi_c
Mal_c + NAD_c <=> NADH_c + OAA_c
Glu_h + Mal_c --> Glu_c + Mal_h
Mal_c + cACN_m <=> Mal_m + cACN_c
KG_c + Mal_h --> KG_h + Mal_c
Mal_c <=> Mal_p
NADPH_c + OAA_c --> Mal_c + NADP_c
Cit_m + Mal_c <=> Cit_c + Mal_m
 
Pi_h + starch2_h --> G1P_h + starch1_h
H2O_h + starch3_h --> Mas_h + starch1_h
ADPG_h + starch1_h --> ADP_h + H_h + starch2_h
ADPG_h + H2O_h --> ADP_h + H_h + starch1_h
Mas_h + starch1_h --> Glc_h + starch2_h
starch1_h --> 
 
2.0 starch3_h --> Glc_h + starch5_h
H2O_h + starch5_h --> Mas_h + starch3_h
H2O_h + starch3_h --> Mas_h + starch1_h
ADPG_h + starch2_h --> ADP_h + H_h + starch3_h
starch3_h --> 
Mas_h + starch2_h --> Glc_h + starch3_h
 
H2O_h + starch5_h --> Mas_h + starch3_h
2.0 starch3_h 

In [20]:
ANmodel = cobra.io.read_sbml_model("ANmodel_Col0_Control.xml")

for m in ['Fum_c','Mal_c',"starch1_h"]:
    metabo = ANmodel.metabolites.get_by_id(m)
    for r in metabo.reactions:
        ind1 = ANmodel.reactions.index(r)
        print(r.reaction)
        print(r.bounds)
        print(ind1)
        print(ANmodel.reactions[ind1])
        print(" ")
    print(" ")

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7fa5be2484a8>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7fa592d27860>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7fa5be248ef0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Fum_m + SCA_c --> Fum_c + SCA_m
(0.0, 1000.0)
355
Tr_SFC: Fum_m + SCA_c --> Fum_c + SCA_m
 
Fum_h --> Fum_c
(0.0, 1000.0)
368
Tr_Fum: Fum_h --> Fum_c
 
Fum_c --> 
(1.188, 1.212)
550
Fum_Store: Fum_c --> 
 
Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
164
FumHA_c: Mal_c --> Fum_c + H2O_c
 
 
Mal_c + cACN_m <=> Mal_m + cACN_c
(-1000.0, 1000.0)
348
Tr_DTC2C: Mal_c + cACN_m <=> Mal_m + cACN_c
 
Mal_c + Pi_m <=> Mal_m + Pi_c
(-1000.0, 1000.0)
339
Tr_DIC1: Mal_c + Pi_m <=> Mal_m + Pi_c
 
H_c + Mal_c + NADP_c --> CO2_c + NADPH_c + Pyr_c
(0.0, 1000.0)
111
MalDH2NADP_c: H_c + Mal_c + NADP_c --> CO2_c + NADPH_c + Pyr_c
 
Glu_h + Mal_c --> Glu_c + Mal_h
(0.0, 1000.0)
342
Tr_DIT2: Glu_h + Mal_c --> Glu_c + Mal_h
 
Mal_c + NAD_c <=> NADH_c + OAA_c
(-1000.0, 1000.0)
51
MalDH1_c: Mal_c + NAD_c <=> NADH_c + OAA_c
 
Mal_c --> 
(0.594, 0.606)
549
Mal_Store: Mal_c --> 
 
NADPH_c + OAA_c --> Mal_c + NADP_c
(0.0, 1000.0)
114
MalDH3_c: NADPH_c + OAA_c --> Mal_c + NADP_c
 
Mal_c + iCit_m <=> Mal_m + iCit_c
(-1000.0

In [14]:
ANmodel = cobra.io.read_sbml_model("ANmodel_Col0_Control.xml")
for r in ANmodel.reactions:
    if max(r.bounds) != 1000.0:
        print(r.reaction)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7fe0d08b8278>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7fe0cfe21550>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7fe0d08b8c18>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> 
Fum_c --> 
starch1_h --> 
